# Leveraging Sentiment Analysis to Improve Airbnb Visualization Maps

## Loading data

In [2]:
import pyspark
# import pymongo
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Datacamp Pyspark Tutorial").config(
    "spark.memory.offHeap.enabled", "true").config("spark.memory.offHeap.size", "10g").getOrCreate()
df = spark.read.csv('./work/data/listings.csv', header=True, escape="\"")
df.show(5, 0)

# client = pymongo.MongoClient("localhost", 27017)

# print(client)

spark.stop()


+------+--------------------------------------------------------------------------+-------+----------------+-------------------+----------------------------------+-----------------+-----------------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-----------+
|id    |name                                                                      |host_id|host_name       |neighbourhood_group|neighbourhood                     |latitude         |longitude        |room_type      |price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|license    |
+------+--------------------------------------------------------------------------+-------+----------------+-------------------+----------------------------------+-----------------+-----------------+---------------+-----+--------------+-----------------+-----------+

In [4]:
# Import the necessary modules
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# https://github.com/RWaltersMA/mongo-spark-jupyter
# master("spark://spark-master:7077") --> If you add this master to the builder the master worker stops ...  :(
# Create a SparkSession
spark = SparkSession \
    .builder \
    .appName("pyspark-notebook2") \
    .config("spark.executor.memory", "1g") \
    .config("spark.mongodb.input.uri", "mongodb://mongo:27017") \
    .config("spark.mongodb.output.uri", "mongodb://mongo:27017") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0") \
    .getOrCreate()

rdd = spark.sparkContext.parallelize(range(1, 100))

print("THE SUM IS HERE: ", rdd.sum())

# Stop the SparkSession
spark.stop()


RuntimeError: Java gateway process exited before sending its port number

In [3]:
# # Import the necessary modules
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import *

# # https://github.com/RWaltersMA/mongo-spark-jupyter
# # master("spark://spark-master:7077") --> If you add this master to the builder the master worker stops ...  :(
# # Create a SparkSession
# spark = SparkSession \
#     .builder \
#     .appName("Spark NLP") \
#     .config("spark.executor.memory", "1g") \
#     .config("spark.mongodb.input.uri", "mongodb://mongo:27017") \
#     .config("spark.mongodb.output.uri", "mongodb://mongo:27017") \
#     .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0") \
#     .master("local[*]") \
#     .config("spark.driver.memory","16G") \
#     .config("spark.driver.maxResultSize", "0") \
#     .config("spark.kryoserializer.buffer.max", "2000M") \
#     .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3") \
#     .getOrCreate()


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark NLP") \
    .master("local[*]") \
    .config("spark.driver.memory", "16G") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3") \
    .getOrCreate()


:: loading settings :: url = jar:file:/Users/matteocirca/opt/anaconda3/envs/sparknlp/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matteocirca/.ivy2/cache
The jars for the packages stored in: /Users/matteocirca/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7fec1599-7dcf-44bc-a48f-116b02626b6d;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.

23/10/12 18:04:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
listings = spark.read.csv('./data/listings.csv', header=True, escape="\"")
listings.show(5, 0)

print("Number of rows: ", listings.count())


+------+--------------------------------------------------------------------------+-------+----------------+-------------------+----------------------------------+-----------------+-----------------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-----------+
|id    |name                                                                      |host_id|host_name       |neighbourhood_group|neighbourhood                     |latitude         |longitude        |room_type      |price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|license    |
+------+--------------------------------------------------------------------------+-------+----------------+-------------------+----------------------------------+-----------------+-----------------+---------------+-----+--------------+-----------------+-----------+

In [4]:
reviews = spark.read.csv('./data/reviews.csv', header=True, escape="\"")
reviews.show(5, 0)

print("Number of rows: ", reviews.count())


+----------------------+--------------------+----------+-----------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|listing_id            |id                  |date      |reviewer_id|reviewer_name|comments                                                                                                                                       |
+----------------------+--------------------+----------+-----------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|18674                 |4808211             |2013-05-27|4841196    |Caron        |Great location. Clean, spacious flat. Would recommend to anyone.                                                                               |
|18674                 |10660311            |2014-03-02|11600277   |Juan Carlos  |Mi mejor r

Number of rows:  845479


In [5]:
# join listings and reviews on the id and listing_id columns
listings_reviews = listings.join(reviews, listings.id == reviews.listing_id)
listings_reviews.show(5, 0)

print("Number of rows: ", listings_reviews.count())


+-----+----------------------------------------------------------------+-------+----------------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-----------+----------+---------+----------+-----------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id   |name                                                            |host_id|host_name

Number of rows:  763592


In [6]:
# keep only id, latitude, longitude, and comments
listings_reviews = listings_reviews.select(
    "listing_id", "latitude", "longitude", "comments")
listings_reviews.show(5, 0)


+----------+--------+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|listing_id|latitude|longitude|comments                                                                                                                                                                                                                                                                                                                                                                                                               

## Cleaning the data

## Sentiment Analysis

In [ ]:
# Import the required modules and classes
from sparknlp.base import DocumentAssembler, Pipeline, LightPipeline
from sparknlp.annotator import (
    UniversalSentenceEncoder,
    SentimentDLModel
)
import pyspark.sql.functions as F

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained("tfhub_use", "en")\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

sentimentdl = SentimentDLModel.pretrained("sentimentdl_use_twitter", "en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        use,
        sentimentdl
    ])

text_list = [
    """@Mbjthegreat i really dont want AT&amp;T phone service..they suck when it comes to having a signal""",
    """holy crap. I take a nap for 4 hours and Pitchfork blows up my twitter dashboard. I wish I was at Coachella.""",
    """@Susy412 he is working today  ive tried that still not working..... hmmmm!! im rubbish with computers haha!""",
    """Brand New Canon EOS 50D 15MP DSLR Camera Canon 17-85mm IS Lens ...: Web Technology Thread, Brand New Canon EOS 5.. http://u.mavrev.com/5a3t""",
    """Watching a programme about the life of Hitler, its only enhancing my geekiness of history.""",
    """GM says expects announcment on sale of Hummer soon - Reuters: WDSUGM says expects announcment on sale of Hummer .. http://bit.ly/4E1Fv""",
    """@accannis @edog1203 Great Stanford course. Thanks for making it available to the public! Really helpful and informative for starting off!""",
    """@the_real_usher LeBron is cool.  I like his personality...he has good character.""",
    """@sketchbug Lebron is a hometown hero to me, lol I love the Lakers but let's go Cavs, lol""",
    """@PDubyaD right!!! LOL we'll get there!! I have high expectations, Warren Buffet style.""",
]


tfhub_use download started this may take some time.


In [ ]:
empty_df = spark.createDataFrame([['']]).toDF("text")

model = nlpPipeline.fit(empty_df)

df = spark.createDataFrame(pd.DataFrame({"text":text_list}))
result = model.transform(df)